In [2]:
import pandas as pd
import numpy as np
import xml.etree
import matplotlib.pyplot as plt
import seaborn as sns
from implementations import *
import csv
import gzip
import xml.etree.ElementTree as ET
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
import nltk
import spacy

# nltk.download('averaged_perceptron_tagger')

In [3]:
df = pd.read_csv('characters_descriptions.csv')
movie_md = load_movie_md()
df1 = pd.read_csv('characters_actions.csv')

In [45]:
df

,actor name,actor id,actor gender,character name,dependency,governor,dependent,action,movie id
0,Richard Cetrone,/m/0bgcht7,M,Big Daddy Mars,dobj,depicts,Mars,depicts,975900
1,Richard Cetrone,/m/0bgcht7,M,Big Daddy Mars,dobj,depicts,Mars,depicts,975900
2,Richard Cetrone,/m/0bgcht7,M,Big Daddy Mars,dobj,depicts,Mars,depicts,975900
3,Natasha Henstridge,/m/0346l4,F,Lieutenant Melanie Ballard,nn,Ballard,Melanie,Ballard,975900
4,Natasha Henstridge,/m/0346l4,F,Lieutenant Melanie Ballard,nn,Ballard,Melanie,Melanie,975900
...,...,...,...,...,...,...,...,...,...
2561037,Tsutomu Takayama,/m/0kr4097,M,Hibiki,nsubj,end,Hibiki,end,913762
2561038,Jonathan Fahn,/m/0bx7_j,M,Hibiki,nsubj,end,Hibiki,end,913762
2561039,笠原弘子,/m/01qwg7h,F,Ishtar,nn,attempt,Ishtar,attempt,913762
2561040,Tsutomu Takayama,/m/0kr4097,M,Hibiki,conj_and,Hibiki,attempt,attempt,913762


In [46]:
def upper(word):
    return word[0].isupper()

df = df[~df['action'].apply(upper)]

In [68]:
nlp = spacy.load("en_core_web_sm")
#doc = nlp('i ' + word)[0].tag_[:2]=='VB'
#doc[0].tag_[:2]

def is_VB2(word):
    return (nlp('i ' + word)[1].tag_[:2]=='VB')&(nlp('he ' + word)[1].tag_[:2]=='VB')


def is_VB(word):
    return (pos_tag(nltk.word_tokenize('he ' + word))[1][1][:2]=='VB')&(pos_tag(nltk.word_tokenize('i ' + word))[1][1][:2]=='VB')

In [78]:
#is_VB2('rowing')

True

In [73]:
# lemmatize verbs and uppercase them
df['action'] = df['action'].apply(str.lower)
df = df[df['action'].apply(is_VB2)]
lemmatizer = WordNetLemmatizer()
df['action'] = df['action'].apply(lambda x: lemmatizer.lemmatize(x,'v'))

In [32]:
df.to_csv('characters_actions.csv', index=False)

In [4]:
df = df1 

In [5]:
irrelevant_words = ['\'s', 'and', 'with', 'of', 'to', 'by', 'for', 'in', 'is', 'he', 'has', 'from', 'as',
                   'his', 'on', 'dr.', 'a', 'john', 'drs.', 'father', 'son', 'friend', 
                   'daughter', 'wife', 'mother', 'family', 'sister', 'brother', 'man', 'men', 'captain', 
                   'singh', 'jack', 'king', 'girl', 'woman', 'girlfriend', 'mary', 'husband', 'parent', 
                   'lady', 'children', 'white', 'car', 'room', 'annie', 'lady', 'queen', 'boy', 'student', 
                    'the', 'friends', 'her', 'home', 'ann', 'anne', 'aunt', 'father', 'girl', 'jane', 
                   'man', 'ms.', 'princess', 'sir', 'sister', 'snow', 'brothers', 'mrs.', 'mrs', 'mr', 
                    'winter', 'apartment', 'mr.', 'widow', 'jim', 'larry', 'van', 'officer',
                   'pregnant', 'teacher', 'relationship', 'team', 'socialite', 'maid', 'distraught', 
                   'susan', 'elizabeth', 'anna', 'l', 'sentence', 'steve', 'charlie', '|', 'heartbroken', 
                   'kitty', 'fred', 'boyfriend', 'eddie', 'wealthy', 'delight', 'wound', 'upset', 
                   'alex', 'de', 'crew', 'harry', 'ryan', 'lisa', 'adam', 'place', 'fire', ]

conditions = (df['action']!='')

for irr_wrd in list(set(irrelevant_words)):
    conditions = conditions&(df['action']!=irr_wrd)
    
act_df_treated = df[conditions]

char_description_total = act_df_treated.merge(movie_md, left_on ='movie id', right_on = 'Wikipedia movie ID')
epochs = [1914, 1960, 2000, 2022]
df_epochs = [char_description_total.query('`Movie release date` >=' + str(epochs[i]+1) + ' and `Movie release date` <= ' + str(epochs[i+1])) for i in range(len(epochs)-1)]

In [7]:
char_description_total

,actor name,actor id,actor gender,character name,dependency,governor,dependent,action,movie id,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres
0,Ice Cube,/m/01vw26l,M,Desolation Williams,nsubj,sent,Williams,send,975900,975900,/m/03vyhn,Ghosts of Mars,2001,14010832.0,98.0,[English Language],[United States of America],"[Thriller, Science Fiction, Horror, Adventure,..."
1,Ice Cube,/m/01vw26l,M,Desolation Williams,nsubjpass,held,Williams,hold,975900,975900,/m/03vyhn,Ghosts of Mars,2001,14010832.0,98.0,[English Language],[United States of America],"[Thriller, Science Fiction, Horror, Adventure,..."
2,Ice Cube,/m/01vw26l,M,Desolation Williams,prep_after,wiped,Williams,wipe,975900,975900,/m/03vyhn,Ghosts of Mars,2001,14010832.0,98.0,[English Language],[United States of America],"[Thriller, Science Fiction, Horror, Adventure,..."
3,Natasha Henstridge,/m/0346l4,F,Lieutenant Melanie Ballard,dep,Ballard,wanting,want,975900,975900,/m/03vyhn,Ghosts of Mars,2001,14010832.0,98.0,[English Language],[United States of America],"[Thriller, Science Fiction, Horror, Adventure,..."
4,John Hawkes,/m/0785v8,M,Richard Swersey,dobj,introducing,Richard,introduce,2238856,2238856,/m/06yc6v,Me and You and Everyone We Know,2005,8012838.0,91.0,[English Language],"[United States of America, United Kingdom]","[Romantic comedy, Indie, Comedy of manners, Co..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195325,Prithviraj Sukumaran,/m/0cqcgj,M,Surya,nsubjpass,arrested,Surya,arrest,24997872,24997872,/m/09g6klx,Pokkiri Raja,2010,NaN,136.0,[Malayalam Language],[India],[Drama]
195326,Prithviraj Sukumaran,/m/0cqcgj,M,Surya,nsubj,is,Surya,be,24997872,24997872,/m/09g6klx,Pokkiri Raja,2010,NaN,136.0,[Malayalam Language],[India],[Drama]
195327,Prithviraj Sukumaran,/m/0cqcgj,M,Surya,prep_with,love,Surya,love,24997872,24997872,/m/09g6klx,Pokkiri Raja,2010,NaN,136.0,[Malayalam Language],[India],[Drama]
195328,Mammootty,/m/02hkv5,M,Raja,prep_to,gives,Raja,give,24997872,24997872,/m/09g6klx,Pokkiri Raja,2010,NaN,136.0,[Malayalam Language],[India],[Drama]


In [36]:
male_description = df_epochs[0][df_epochs[0]['actor gender']=='M']
female_description = df_epochs[0][df_epochs[0]['actor gender']=='F']
#head_males = male_description.groupby('action', as_index=False).count().sort_values('actor name', ascending = False).head(45)
#head_females = female_description.groupby('action', as_index=False).count().sort_values('actor name', ascending = False).head(45)

In [37]:
m = male_description.groupby('action', as_index = False).count()[['action', 'actor name']]
m['actor name'] *= 100/male_description.shape[0]
f = female_description.groupby('action', as_index = False).count()[['action', 'actor name']]
f['actor name'] *= 100/female_description.shape[0]
merged_df = m.merge(f, on = 'action', suffixes=('_m', '_f'))
merged_df['freq_diff'] = merged_df['actor name_m'] - merged_df['actor name_f']
merged_df['freq_ratio'] = merged_df['actor name_m']/merged_df['actor name_f']
print('male actions size is    : {}'.format(male_description.shape[0]))
print('female actions size is  : {}'.format(female_description.shape[0]))

male actions size is    : 12511
female actions size is  : 5825


In [30]:
merged_df.sort_values('freq_ratio').query('`actor name_f` > 0.15 or `actor name_m` > 0.15').head(30)

,action,actor name_m,actor name_f,freq_diff,freq_ratio
1107,prostitute,0.022920,0.218479,-0.195559,0.104907
853,kiss,0.089770,0.262175,-0.172405,0.342405
916,marry,0.551990,1.279097,-0.727107,0.431547
901,love,0.509970,1.096369,-0.586399,0.465145
1358,sleep,0.118420,0.250258,-0.131838,0.473191
528,engage,0.110780,0.230396,-0.119616,0.480824
889,live,0.223470,0.460793,-0.237323,0.484969
124,attract,0.150890,0.301899,-0.151009,0.499803
951,move,0.242570,0.460793,-0.218223,0.526419
1305,seduce,0.110780,0.202590,-0.091810,0.546819


In [31]:
merged_df.sort_values('freq_ratio', ascending = False).query('`actor name_f` > 0.15 or `actor name_m` > 0.15').head(30)

,action,actor name_m,actor name_f,freq_diff,freq_ratio
148,beat,0.282680,0.107254,0.175427,2.635626
96,arrest,0.456490,0.194645,0.261845,2.345242
869,lead,0.511880,0.262175,0.249705,1.952435
1467,suspect,0.175720,0.091364,0.084356,1.923294
1445,succeed,0.175720,0.095336,0.080384,1.843157
851,kill,1.634961,0.917613,0.717347,1.781753
109,assign,0.212010,0.119171,0.092840,1.779047
979,offer,0.162350,0.091364,0.070986,1.776957
736,hire,0.173810,0.103281,0.070529,1.682883
836,investigate,0.152800,0.091364,0.061436,1.672430
